Входной файл .xls список аккредитованных ИТ компаний по России. 
Каждый элемент списка не имеет адреса регистрации.
Задача: Добавить адрес и отфильтровать для Красноряского края




In [22]:
from google.colab import drive
!pip install pandas
!pip install --upgrade xlrd
import pandas as pd
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

drive.mount('/content/drive')

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [5]:
ex_data = pd.read_excel('/content/drive/MyDrive/rubbish/reestr-akkreditovannyx-10022022.xls')
p = pd.DataFrame(ex_data)

In [56]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [57]:
#В google.colab время выполнения около 5ч для ~15тыс компаний
url = 'https://star-pro.ru/proverka-kontragenta'
driver.get(url)
driver.set_window_size(1920,1080)
adress_list = []
for i in p['Идентификационный номер налогоплательщика']:

    driver.find_element(By.ID, 'org-search-input').send_keys(str(i))
    driver.find_element(By.ID, 'org-search-input').send_keys(Keys.ENTER)
    adress = driver.find_element(By.CLASS_NAME, 'ftfbn-venue__address').text
    driver.find_element(By.ID, 'org-search-input').clear()
    adress_list.append(adress)

# driver.save_screenshot('image.png')
driver.close()

adress_list

['603093, ОБЛАСТЬ НИЖЕГОРОДСКАЯ, , ГОРОД НИЖНИЙ НОВГОРОД, , УЛИЦА РОДИОНОВА, 23, ,',
 '347900, ОБЛАСТЬ РОСТОВСКАЯ, , , ГОРОД ТАГАНРОГ, , , УЛИЦА ИНСТРУМЕНТАЛЬНАЯ, 29-2, , ,',
 '129626, ГОРОД МОСКВА, , , , , , ПЕРЕУЛОК 1-Й РИЖСКИЙ, ДОМ 3, , ,',
 '143402, ОБЛАСТЬ МОСКОВСКАЯ, , ГОРОД КРАСНОГОРСК, , УЛИЦА ЖУКОВСКОГО, 17, , 3.9.11',
 '680014, КРАЙ ХАБАРОВСКИЙ, , ГОРОД ХАБАРОВСК, , УЛИЦА БОЛЬШОЙ АЭРОДРОМ, 10, , 32',
 '117420, ГОРОД МОСКВА, , , , УЛИЦА ПРОФСОЮЗНАЯ, 57, ,',
 '127006, ГОРОД МОСКВА, , , , УЛИЦА ДОЛГОРУКОВСКАЯ, ДОМ 40, , ЭТ 2 ПОМ IV КАБ 7',
 '603074, ОБЛАСТЬ НИЖЕГОРОДСКАЯ, , ГОРОД НИЖНИЙ НОВГОРОД, , ПРОЕЗД БУРНАКОВСКИЙ, 15, 33, 906',
 '198504, ГОРОД САНКТ-ПЕТЕРБУРГ, , , ГОРОД ПЕТЕРГОФ, , , УЛИЦА ЧИЧЕРИНСКАЯ, ДОМ 2, ЛИТЕРА А, , ПОМЕЩЕНИЕ 5-Н',
 '170024, ОБЛ. ТВЕРСКАЯ, , Г. Тверь, , ПР-КТ НИКОЛАЯ КОРЫТКОВА, Д. 3А, ,',
 '120220, ГОРОД МОСКВА, , , , ПЕРЕУЛОК ДЕНЕЖНЫЙ, ДОМ 21, ,',
 '300041, ОБЛАСТЬ ТУЛЬСКАЯ, , ГОРОД ТУЛА, , УЛИЦА СОВЕТСКАЯ, ДОМ 112, ,',
 '125040, ГОРОД МОСКВА, , , , У

In [68]:
with open('adress.txt', 'w+') as w:
  w.writelines(map(lambda x: x + '\n', adress_list))

In [70]:
a = [x.split(', ') for x in adress_list]

In [92]:
#выделил списки по столбцам для преобразования в df
b = [[a[x][y] for x in range(len(adress_list))] for y in range(len(a[0])) ]

In [153]:
kray = pd.Series(b[1])
table = {'name': p['Полное наименование организации'],
         'kray': kray,
         'city': pd.Series(b[3]),
         '4': pd.Series(b[4]),
         '5': pd.Series(b[5]),
         '6':pd.Series(b[6]),
         "7":pd.Series(b[7]),
         }
df = pd.DataFrame(table)

In [150]:
from collections import Counter 
# df[df.kray == 'Красноярск']
Counter(df.kray)

Counter({'А.ОБЛ. ЕВРЕЙСКАЯ': 2,
         'А.ОКР. ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА': 6,
         'АВТОНОМНАЯ ОБЛАСТЬ ЕВРЕЙСКАЯ': 3,
         'АВТОНОМНЫЙ ОКРУГ НЕНЕЦКИЙ': 4,
         'АВТОНОМНЫЙ ОКРУГ ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА': 54,
         'АВТОНОМНЫЙ ОКРУГ ЧУКОТСКИЙ': 1,
         'АВТОНОМНЫЙ ОКРУГ ЯМАЛО-НЕНЕЦКИЙ': 6,
         'Г. МОСКВА': 1441,
         'Г. САНКТ-ПЕТЕРБУРГ': 434,
         'Г. СЕВАСТОПОЛЬ': 5,
         'ГОРОД БАЙКОНУР': 1,
         'ГОРОД МОСКВА': 4706,
         'ГОРОД САНКТ-ПЕТЕРБУРГ': 1625,
         'ГОРОД СЕВАСТОПОЛЬ': 31,
         'Город Москва': 1,
         'Город Севастополь': 1,
         'КРАЙ АЛТАЙСКИЙ': 83,
         'КРАЙ ЗАБАЙКАЛЬСКИЙ': 9,
         'КРАЙ КАМЧАТСКИЙ': 8,
         'КРАЙ КРАСНОДАРСКИЙ': 270,
         'КРАЙ КРАСНОЯРСКИЙ': 130,
         'КРАЙ ПЕРМСКИЙ': 243,
         'КРАЙ ПРИМОРСКИЙ': 69,
         'КРАЙ СТАВРОПОЛЬСКИЙ': 66,
         'КРАЙ ХАБАРОВСКИЙ': 62,
         'Край Камчатский': 2,
         'Край Краснодарский': 1,
   

In [151]:
df[df.kray == 'КРАЙ КРАСНОЯРСКИЙ'].to_excel('Krasnoyarsk.xlsx')